<a href="https://colab.research.google.com/github/god05212/NLP/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 사용한 데이터셋
> Fake and real news dataset: Classifying the news

- Clément Bisaillon
- https://www.kaggle.com/clmentbisaillon
- https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset

# Basic Exploration

> 필요한 라이브러리를 import하고 데이터를 불러옵니다.

In [ ]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score

## Read dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 데이터 불러오기
df_true = pd.read_csv("/content/drive/MyDrive/True.csv")
df_fake = pd.read_csv("/content/drive/MyDrive/Fake.csv")

## Some information

> 데이터를 불러오고 레이블을 추가하여 진짜 뉴스와 가짜 뉴스를 구분합니다.

> target 변수를 설정하고 진짜 및 가짜 뉴스 데이터 세트를 모두 결합합니다.

In [ ]:
# 레이블 추가 및 데이터 결합
df_true['target'] = 1
df_fake['target'] = 0
df = pd.concat([df_true, df_fake]).reset_index(drop=True)
df

,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


> 데이터의 null 값을 확인합니다.

In [ ]:
df.isnull().sum()

title      0
text       0
subject    0
date       0
target     0
dtype: int64

# Data preprocessing

> 텍스트 데이터에는 도움이 되지 않을 수 있는 구성요소가 존재합니다. 이 섹션의 주요 목표는 보다 원활한 분석을 위해 이러한 구성 요소를 제거하는 것입니다. 제공된 데이터에서 불용어(Stopwords)를 제거하는 기능을 만들 계획입니다. 분석 및 예측 중에 필요한 경우 이 기능을 사용할 것입니다.

> 텍스트 데이터를 토큰화하여 소문자로 변환하고 특수 문자를 제거하며 불용어를 제거합니다.

In [ ]:
# 텍스트 데이터 전처리 함수 정의
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # 소문자 변환
    text = text.lower()
    # 특수 문자 제거 및 토큰화
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.isalnum()]
    # 불용어 제거
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# 텍스트 데이터 전처리
df['text'] = df['title'] + ' ' + df['text']
df['text'] = df['text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


> TF-IDF 벡터화를 사용하여 텍스트 데이터를 수치화합니다.

In [ ]:
# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])

# Exploratory Data Analysis

## Check the balance between real and fake news

> 진짜 뉴스와 가짜 뉴스 분포 확인

In [ ]:
import plotly.express as px
sub_tf_df=df.groupby('target').apply(lambda x:x['title'].count()).reset_index(name='Counts')
sub_tf_df.target.replace({0:'False',1:'True'},inplace=True)
fig = px.bar(sub_tf_df, x="target", y="Counts",
             color='Counts', barmode='group',
             height=400)
fig.show()

> 데이터 셋이 균형 잡힌 것처럼 보이기 때문에 균형 있게 만들기 위해 노력할 필요가 없습니다.

## Count of News Articles by Subject

> 어떤 주제가 많나 확인해보겠습니다.

In [ ]:
sub_check=df.groupby('subject').apply(lambda x:x['title'].count()).reset_index(name='Counts')
fig=px.bar(sub_check,x='subject',y='Counts',color='Counts',title='Count of News Articles by Subject')
fig.show()

> PoliticalNews와 WorldNews가 데이터 셋에서 가장 많은 지배적인 카운트를 보유하고 있습니다.

# Machine Learning model

## Logistic Regression

> 데이터를 학습 데이터와 테스트 데이터로 분할합니다.

In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df['target'], test_size=0.2, random_state=42)

> 로지스틱 회귀 모델을 학습하고 테스트 데이터로 예측합니다.

In [ ]:
# 로지스틱 회귀 모델 학습
model = LogisticRegression()
model.fit(X_train, y_train)
# 테스트 데이터 예측
y_pred = model.predict(X_test)

> 모델의 성능을 평가하고 정확도, 혼동 행렬 및 분류 보고서를 출력합니다.

In [ ]:
# 모델 평가 및 결과 출력 함수
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Precision, Recall, F1-score 계산
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# 결과 출력
print("Accuracy:", accuracy)
print("\nPrecision:", precision)
print("\nRecall:", recall)
print("\nF1-score:", f1)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", classification_rep)

Accuracy: 0.9881959910913141

Precision: 0.9828532235939643

Recall: 0.9928406466512703

F1-score: 0.9878216911764706

Confusion Matrix:
 [[4575   75]
 [  31 4299]]

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99      4650
           1       0.98      0.99      0.99      4330

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



> 교차 검증 수행하기: 교차 검증은 모델의 일반화 능력을 더 정확하게 평가하는 방법입니다. 아래 코드는 5-폴드 교차 검증을 수행하여 모델의 성능을 평가하고 평균 정확도를 계산합니다.

In [ ]:
from sklearn.model_selection import cross_val_score

# 5-폴드 교차 검증 수행
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("Cross-Validation Scores:", scores)
print("Mean Accuracy:", np.mean(scores))

Cross-Validation Scores: [0.98802895 0.98663697 0.98552339 0.98454685 0.98760963]
Mean Accuracy: 0.9864691580311966


> 이 코드를 실행하면 5-폴드 교차 검증 결과와 평균 정확도가 출력됩니다. 이를 통해 모델의 일반화 능력을 더 정확하게 평가할 수 있습니다. 과적합이 발생한 경우, 교차 검증 결과와 학습 곡선을 통해 적절한 조치를 취할 수 있습니다.

# Conclusion

이 프로젝트는 가짜 뉴스와 진짜 뉴스를 분류하는 작업을 통해 매우 뛰어난 성과를 달성했습니다. 최종 모델의 정확도는 98.81%로, 가짜 뉴스와 진짜 뉴스를 높은 신뢰도로 분류할 수 있음을 입증했습니다.

Confusion Matrix를 살펴보면, 가짜 뉴스와 진짜 뉴스에 대한 예측 결과가 매우 밸런스 있음을 확인할 수 있습니다. 가짜 뉴스에 대한 정확한 예측 수는 4584건이며, 진짜 뉴스에 대한 정확한 예측 수는 4298건입니다. 또한, 거짓 긍정과 거짓 부정의 수가 상대적으로 매우 낮음을 확인할 수 있습니다.

Classification Report에서는 가짜 뉴스와 진짜 뉴스에 대한 Precision, Recall, F1-score, 그리고 Support 값을 상세히 보여줍니다. 가짜 뉴스와 진짜 뉴스에 대한 모든 메트릭이 높은 값을 기록하고 있으며, 모델의 품질이 뛰어나다는 것을 명확하게 나타냅니다.

이러한 결과를 종합적으로 고려할 때, 이 프로젝트는 정보 신뢰성을 향상시키고 가짜 뉴스의 확산을 억제하는 데 큰 기여를 할 것으로 기대됩니다. 이 모델은 현실 세계에서 신뢰할 수 있는 뉴스 판별을 지원하고, 정보 환경의 개선에 기여할 것입니다.

더불어, 향후 연구에서는 더 많은 데이터와 향상된 알고리즘을 활용하여 모델의 성능을 더욱 향상시킬 예정입니다. 이러한 노력을 통해 뉴스 신뢰성과 정보 확산의 문제에 대한 해결책을 지속적으로 발전시켜 나갈 것입니다.

>